# 2. Wrangle the Raster Data (3 layers)
# Part 3: MACA v2 THREDDS - download 2 climate scenarios

In [28]:
# Download stored variables from previous notebooks

# Variables from study area download
%store -r habitat_suitability_data_dir usfs_grasslands_path 
%store -r comanche_grassland_gdf pawnee_grassland_gdf usfs_grasslands_gdf

# Variables from Polaris download


# Variables from SRTM download
#%store srtm_comanche_result srtm_pawnee_result slope_da slope_list

In [29]:
# Prepare for download Part 1 of 1
## Import packages that will help with...

# Reproducible file paths
import os # Reproducible file paths
import pathlib # Find the home folder

# Find files by pattern
import matplotlib.pyplot as plt # Overlay pandas (vector) and xarry (raster) plots 
import rioxarray as rxr # Work with geospatial raster data

# Work with tabular, vector, and raster data
import geopandas as gpd # work with vector data
import pandas as pd # Group and aggregate
import xarray as xr # Adjust images

In [37]:
# Create function  that converts longitude that is in the 0-360 
# range, to the -180 to 180 range 
def convert_longitude(longitude):
    """ Convert logitude range from  0-360 to -180-180"""
    return (longitude - 360) if longitude > 180 else longitude

# Create list to save data arrays back to
maca_da_list = []

# Iterate through multiple sites or study areas
for site_name, site_gdf in {
    'comanche':comanche_grassland_gdf,'pawnee':pawnee_grassland_gdf}.items():
    # Iterate through mutliple models, e.g. CCSM4
    for model in ['CCSM4']:
        # Iterate through multiple variables, e.g. precipitation
        for variable in ['pr']:
            # Iterate through different emissions scenarios
            for scenario in ['historical', 'rcp85']:
                # Iterate through start years and end years
                for scenario_names, start_year in {
                    'historical': 2000, 'rcp85': 2096}.items():
                    end_year = start_year + 4

                    # Define template url for MACA v2 download
                    maca_url = (
                        'http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2'
                        f'/CCSM4/macav2metdata_{variable}_CCSM4_r6i1p1_'
                        f'{scenario}_{start_year}_{end_year}_CONUS_monthly.nc')
                    # Connect to the raster image
                    maca_da = xr.open_dataset(maca_url).squeeze().precipitation
                    # Get the study bounds
                    bounds = site_gdf.to_crs(maca_da.rio.crs).total_bounds
                    # Apply function convert_longitude to convert longitude
                    maca_da = maca_da.assign_coords(
                        lon = ('lon', 
                        [convert_longitude(l) for l in maca_da.lon.values]))
                    # Set spatial dimensions - need lon = x-axis and lat = y-axis.
                    maca_da = maca_da.rio.set_spatial_dims(x_dim='lon', y_dim='lat')
                    # Crop the raster image to the bounds of the study area(s)
                    maca_da = maca_da.rio.clip_box(*bounds)
                    # Append the data array to the list
                    maca_da_list.append(dict(
                        site_name = site_name,
                        variable = variable,
                        scenario = scenario_names,
                        start_year = start_year,
                        da = maca_da
                        ))           
# Convert maca_da_list to df, call maca_df to see it
#maca_df = pd.DataFrame(maca_da_list)
#maca_df

OSError: [Errno -90] NetCDF: file not found: 'http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/CCSM4/macav2metdata_pr_CCSM4_r6i1p1_historical_2096_2100_CONUS_monthly.nc'

In [ ]:
# Create function  that converts longitude that is in the 0-360 
# range, to the -180 to 180 range 
def convert_longitude(longitude):
    """ Convert logitude range from  0-360 to -180-180"""
    return (longitude - 360) if longitude > 180 else longitude

maca_url = ('http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2'
'/CCSM4/macav2metdata_pr_CCSM4_r6i1p1_rcp85_2096_2099_CONUS_monthly.nc'
)

# Connect to the raster image
maca_da = xr.open_dataset(maca_url).squeeze().precipitation

# Get the study bounds
bounds = comanche_grassland_gdf.to_crs(maca_da.rio.crs).total_bounds
# Apply function convert_longitude to convert longitude
maca_da = maca_da.assign_coords(
    lon = ('lon', 
    [convert_longitude(l) for l in maca_da.lon.values]))
# Set spatial dimensions - need lon = x-axis and lat = y-axis.
maca_da = maca_da.rio.set_spatial_dims(x_dim='lon', y_dim='lat')
# Crop the raster image to the bounds of the study area(s)
maca_da = maca_da.rio.clip_box(*bounds)
# Append the data array to the list
#maca_da.append(maca_da)

maca_da

In [ ]:
# Iterate through different emissions scenarios
            for scenario in ['historical', 'rcp85']:
                # Iterate through start years and end years
                for scenario_names, start_year in {
                    'historical': 2000, 'rcp85': 2096}.items():
                    end_year = start_year + 4

                    # Define template url for MACA v2 download
                    maca_url = (
                        'http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2'
                        f'/CCSM4/macav2metdata_{variable}_CCSM4_r6i1p1_'
                        f'{scenario}_{start_year}_{end_year}_CONUS_monthly.nc')